In [41]:
from pathlib import Path
import tomli as tomllib
import pandas as pd
import numpy as np

In [39]:
from transit import (
    model_run_dir,
    output_transit_dir,
    read_dbf_and_groupby_sum,
    transit_assignment_filepaths,
    read_transit_assignments,
    time_periods,
    transit_line_rename_filepath,
    transit_validation_2019_alfaro_filepath
    
)

In [3]:
data = read_transit_assignments(model_run_dir, time_periods)

Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEA.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAAM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAMD.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAPM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEV.DBF


In [4]:
def read_nodes(model_run_dir):
    filepath = Path(model_run_dir) / "nodes.xls"
    return pd.read_excel(
        filepath, header=None, names=["Node", "Node Name"]
    )

In [5]:
def station_name():
    station_name = {
        "Station": [
            "12TH",
            "16TH",
            "19TH",
            "24TH",
            "ANTC",
            "ASHB",
            "BALB",
            "BAYF",
            "CAST",
            "CIVC",
            "COLM",
            "COLS",
            "CONC",
            "DALY",
            "DBRK",
            "DELN",
            "DUBL",
            "EMBR",
            "FRMT",
            "FTVL",
            "GLEN",
            "HAYW",
            "LAFY",
            "LAKE",
            "MCAR",
            "MLBR",
            "MONT",
            "NBRK",
            "NCON",
            "OAKL",
            "ORIN",
            "PCTR",
            "PHIL",
            "PITT",
            "PLZA",
            "POWL",
            "RICH",
            "ROCK",
            "SANL",
            "SBRN",
            "SFIA",
            "SHAY",
            "SSAN",
            "UCTY",
            "WARM",
            "WCRK",
            "WDUB",
            "WOAK",
        ],
        "Node": [
            16509,
            16515,
            16508,
            16516,
            15231,
            16525,
            16518,
            16530,
            16537,
            16514,
            16539,
            16532,
            16501,
            16519,
            16523,
            16521,
            16538,
            16511,
            16526,
            16533,
            16517,
            16529,
            16504,
            16534,
            16507,
            16543,
            16512,
            16524,
            16535,
            16000,
            16505,
            15230,
            16502,
            16536,
            16522,
            16513,
            16520,
            16506,
            16531,
            16541,
            16542,
            16528,
            16540,
            16527,
            16544,
            16503,
            16545,
            16510,
        ],
    }
    df_station_name = pd.DataFrame(station_name)
    return df_station_name

In [6]:
def process_BART_data(file_name, model_run_dir):
    # Process BART data for different routes and columns
    nodes = read_nodes(model_run_dir)
    station = station_name()
    BART_BRDA = read_dbf_and_groupby_sum(file_name, "BART", ["A","TOD"], "AB_BRDA")
    EBART_BRDA = read_dbf_and_groupby_sum(file_name, "EBART", ["A", "TOD"], "AB_BRDA")
    OAC_BRDA = read_dbf_and_groupby_sum(file_name, "OAC", ["A", "TOD"], "AB_BRDA")

    BART_XITA = read_dbf_and_groupby_sum(file_name, "BART", ["A", "TOD"], "AB_XITA")
    EBART_XITA = read_dbf_and_groupby_sum(file_name, "EBART", ["A", "TOD"], "AB_XITA")
    OAC_XITA = read_dbf_and_groupby_sum(file_name, "OAC", ["A", "TOD"], "AB_XITA")

    # Concatenate and group data
    BART_A = pd.concat([BART_BRDA, EBART_BRDA, OAC_BRDA])
    BART_A = BART_A.groupby(["A", "TOD"])["AB_BRDA"].sum().reset_index()
    BART_A.columns = ["Node", "TOD", "AB_BRDA"]

    BART_B = pd.concat([BART_XITA, EBART_XITA, OAC_XITA])
    BART_B = BART_B.groupby(["A", "TOD"])["AB_XITA"].sum().reset_index()
    BART_B.columns = ["Node", "TOD", "AB_XITA"]

    # Merge with other dataframes
    BART_A = pd.merge(BART_A, nodes, on=["Node"], how="left")
    BART_A = pd.merge(BART_A, station, on=["Node"], how="right")
    BART = pd.merge(BART_A, BART_B, on=["Node", "TOD"], how="right")

    # Drop rows with specific values
    values_to_drop = ["Hillcrest eBART", "Coliseium OAC", "Somersville Road eBART"]
    BART = BART[~BART["Node Name"].isin(values_to_drop)]

    # Add columns and rearrange columns
    BART["Key"] = BART["Station"] + BART["TOD"]
    BART.columns = [
        "Node",
        "TOD",
        "Boardings",
        "Node Name",
        "Station",
        "Alightings",
        "Key",
    ]
    BART = BART[
        ["Node", "Node Name", "Station", "TOD", "Key", "Boardings", "Alightings"]
    ]

    # Sort and reset index
    BART = BART[["Station", "TOD", "Key", "Boardings", "Alightings"]]
    BART = BART.sort_values(by="Key").reset_index(drop=True)

    return BART


In [7]:
def map_station_to_county(station):
    counties = {
        "San Francisco": ["EMBR", "CIVC", "24TH", "MONT", "POWL", "GLEN", "16TH", "BALB"],
        "San Mateo": ["DALY", "COLM", "SSAN", "SBRN", "SFIA", "MLBR"],
        "Contra Costa": [
            "RICH",
            "ORIN",
            "LAFY",
            "WCRK",
            "CONC",
            "NCON",
            "PITT",
            "ANTC",
            "DELN",
            "PHIL",
            "PCTR",
            "PLZA",
        ],
        "Alameda": [
            "WOAK",
            "12TH",
            "19TH",
            "MCAR",
            "ASHB",
            "DUBL",
            "WDUB",
            "CAST",
            "WARM",
            "UCTY",
            "SHAY",
            "HAYW",
            "BAYF",
            "SANL",
            "OAKL",
            "COLS",
            "FTVL",
            "LAKE",
            "ROCK",
            "DBRK",
            "NBRK",
            "FRMT",
        ],
        "Santa Clara": [],  # Add stations for Santa Clara if available
    }
    for county, stations in counties.items():
        if station in stations:
            return county
    return None 

def process_BART_county(file_name, model_run_dir):
    BART_county = process_BART_data(file_name, model_run_dir)
    

    # Add the 'County' column to the DataFrame
    BART_county["County"] = BART_county["Station"].apply(lambda x: map_station_to_county(x))
    BART_county = (
        BART_county.groupby(["County", "TOD"])[["Boardings", "Alightings"]]
        .sum()
        .reset_index()
    )
    return BART_county

In [13]:
def process_BART_SL_data(file_name, A, B):
    # Read the DBF file and group by 'A' and 'B' while summing 'AB_VOL'
    BART_SL = read_dbf_and_groupby_sum(file_name, "BART", ["A", "B", "TOD"], "AB_VOL")

    # Filter rows for IB (16510 to 16511)
    IB = BART_SL[(BART_SL["A"] == A) & (BART_SL["B"] == B)].copy()
    IB["Direction"] = "IB"

    # Filter rows for OB (16511 to 16510)
    OB = BART_SL[(BART_SL["A"] == B) & (BART_SL["B"] == A)].copy()
    OB["Direction"] = "OB"

    # Concatenate IB and OB DataFrames
    result = pd.concat([IB, OB])

    return result


def BART_SL_Concat(file_name, A, B, Screenline):
    # Concatenate the DataFrames
    BART_SL_CT = process_BART_SL_data(file_name, A, B)

    # Add the 'Screenline' column with 'Countyline'
    BART_SL_CT["Screenline"] = Screenline

    # Create the 'Key' column by combining 'Screenline', 'Direction', and 'TOD'
    BART_SL_CT["Key"] = (
        BART_SL_CT["Screenline"] + BART_SL_CT["Direction"] + BART_SL_CT["TOD"]
    )

    # Rename the 'AB_VOL' column to 'Ridership'
    BART_SL_CT = BART_SL_CT.rename(columns={"AB_VOL": "Ridership"})

    # Select and reorder columns
    BART_SL_CT = BART_SL_CT[["Screenline", "Direction", "TOD", "Key", "Ridership"]]

    # Sort by 'Key' and reset the index
    BART_SL_CT = BART_SL_CT.sort_values(by="Key").reset_index(drop=True)

    return BART_SL_CT


In [17]:
transbay_node = [16510, 16511]  # 16510 in, 16511 out
BART_sl_tb = BART_SL_Concat(data, transbay_node[0], transbay_node[1], 'Transbay')
countyline_node = [16519, 16518]  # 16519n-- in, 16518 --out
BART_sl_ct = BART_SL_Concat(data, countyline_node[0], countyline_node[1], "Countyline")

In [29]:
def relevant_station():
    station_locations = {
    "downtown": ["CIVC", "POWL", "MONT", "EMBR"],
    "not_downtown": ["GLEN", "BALB", "24TH", "16TH"],
    }

    # Create station to screenline mapping
    station_to_label = {}
    for loc in station_locations:
        stations = station_locations[loc]
        labels = [loc] * len(station_locations[loc])
        station_to_label = station_to_label | dict(zip(stations, labels))

    relevant_stations = station_locations["downtown"] + station_locations["not_downtown"]
    return relevant_stations
    

    
    

def determineSL(x):
    if (x["A_SL"] == "not_downtown") & (x["B_SL"] == "downtown"):
        return "IB"
    elif (x["A_SL"] == "downtown") & (x["B_SL"] == "not_downtown"):
        return "OB"
    else:
        return False

In [27]:
def process_BART_SF(filename, model_run_dir):
    nodes = read_nodes(model_run_dir)
    df_station_name = station_name()
    relevant_stations = relevant_station()
    lines = ["BART", "EBART", "OAC"]
    
    # Read, group, and sum data for each line
    dfs = [read_dbf_and_groupby_sum(filename, line, ["A", "B", "TOD"], "AB_VOL") for line in lines]

    # Concatenate results
    intra = pd.concat(dfs)

    # Mapping from Node and Station DataFrames
    node_mapping = nodes.set_index("Node")["Node Name"].to_dict()
    station_mapping = df_station_name.set_index("Node")["Station"].to_dict()

    # Apply mappings
    intra["A_name"] = intra["A"].map(node_mapping)
    intra["B_name"] = intra["B"].map(node_mapping)
    intra["A_station"] = intra["A"].map(station_mapping)
    intra["B_station"] = intra["B"].map(station_mapping)

    # Filter out specific values and stations
    values_to_drop = ["Hillcrest eBART", "Coliseium OAC", "Somersville Road eBART"]
    intra = intra[
        ~intra["A_name"].isin(values_to_drop)
        & ~intra["B_name"].isin(values_to_drop)
    ]
    intra = intra[
        intra["A_station"].isin(relevant_stations)
        & intra["B_station"].isin(relevant_stations)
    ]

    # Apply custom station labeling and direction determination
    intra["A_SL"] = intra["A_station"].apply(lambda x: station_to_label[x])
    intra["B_SL"] = intra["B_station"].apply(lambda x: station_to_label[x])
    intra["Direction"] = intra.apply(lambda x: determineSL(x), axis=1)
    intra = intra[intra["Direction"] != False]

    # Set time of day and select final columns
    intra = intra[["Direction", "TOD", "AB_VOL"]]
    intra['Screenline'] = 'Intra-SF'
    intra['Key'] = intra['Screenline'] + intra['Direction'] + intra['TOD']
    intra.columns = ['Direction', 'TOD', 'Ridership', 'Screenline', 'Key']
    intra = intra[['Screenline', 'Direction', 'TOD', 'Key', 'Ridership']]
    intra = intra.sort_values(by=['Direction','TOD'] ).reset_index(drop=True)
    

    return intra

In [32]:
def process_BART_SL(file_name, model_run_dir):
    transbay_node = [16510, 16511]  # 16510 in, 16511 out
    BART_sl_tb = BART_SL_Concat(file_name, transbay_node[0], transbay_node[1], 'Transbay')
    countyline_node = [16519, 16518]  # 16519n-- in, 16518 --out
    BART_sl_ct = BART_SL_Concat(file_name, countyline_node[0], countyline_node[1], "Countyline")
    BART_sf = process_BART_SF(file_name, model_run_dir)
    BART_SL = pd.concat([BART_sl_tb, BART_sl_ct, BART_sf], ignore_index=True)
    return BART_SL

In [36]:
def read_transit_lines(model_run_dir, transit_line_rename_filepath):
    line_names = pd.read_csv(
        Path(model_run_dir) / "transitLineToVehicle.csv",
        usecols=["Name", "Line", "System"],
    )
    line_names = line_names[line_names["System"] == "SF MUNI"]
    line_names = line_names[["Name", "Line"]]

    # TODO simplify this logic; it shouldn't require such computationally intensive
    # creation of Serieses and then dicts... Seems like a polars join then using
    # when/then/otherwise would be much simpler and efficient
    rename = pd.read_csv(transit_line_rename_filepath)
    rename["new_name"] = rename["New"].str.extract(r"(\d+[A-Za-z]*)")
    name_to_trn_asgn_new = pd.Series(
        rename.Trn_asgn_new.values, index=rename.NAME
    ).to_dict()
    new_name_to_line = pd.Series(
        rename.new_name.values, index=rename.Trn_asgn_new
    ).to_dict()

    # Map the Name & Line in line_names to transit_line_rename using the mappings
    line_names["Name"] = (
        line_names["Name"].map(name_to_trn_asgn_new).fillna(line_names["Name"])
    )
    line_names["Line"] = (
        line_names["Name"].map(new_name_to_line).fillna(line_names["Line"])
    )
    return line_names


def transform_line(line):
    if pd.isna(line):
        return np.nan
    elif line.isdigit():  # Case 1: Only numbers
        return int(line)
    elif any(char.isdigit() for char in line) and any(
        char.isalpha() for char in line
    ):  # Case 2: Numbers and letters
        if "SHORT" in line:
            line = line.replace("SHORT", "R")
        if len(line) < 4:
            line = line.zfill(4)
        return line
    elif line.isalpha():  # Case 3: Only letters
        special_names = {
            "J": "J-Church ",
            "K": "KT-Ingleside/Third Street ",
            "L": "L-Taraval ",
            "M": "M-Ocean View ",
            "N": "N-Judah ",
        }
        return special_names.get(line, line)
    else:
        return line


def map_name_to_direction(name):
    if name.endswith("I"):
        return "IB"
    elif name.endswith("O"):
        return "OB"
    else:
        return None  # Return None for other cases

In [53]:
def muni(model_run_dir, transit_line_rename_filepath, transit_validation_2019_alfaro_filepath, output_transit_dir):
    line_names = read_transit_lines(model_run_dir, transit_line_rename_filepath)

    MUNI = read_dbf_and_groupby_sum(data, "SF MUNI", ["FULLNAME", "NAME","TOD"], "AB_BRDA")
    MUNI['Direction'] = MUNI['NAME'].apply(map_name_to_direction)

    MUNI = MUNI.sort_values(by="FULLNAME").reset_index(drop=True)
    MUNI = MUNI.rename(columns={"NAME": "Name", "AB_BRDA": "Ridership"})

    MUNI_full = pd.merge(MUNI, line_names, on="Name", how="left")

    # Apply the transformation function to the 'Line' column
    MUNI_full["Line"] = MUNI_full["Line"].apply(transform_line)
    # TODO make a standard format for MUNI observed data instead of having the code
    # read from the bespoke "Transit_Validation_2019 - MA.xlsx" file
    obs_MUNI_line = pd.read_excel(
        transit_validation_2019_alfaro_filepath,
        usecols="B:H",
        sheet_name="obs_MUNI_line",
        skiprows=list(range(9)),
    )
    mode = obs_MUNI_line[["Line", "Mode"]].drop_duplicates().reset_index(drop=True)
    mode_dict = mode.set_index("Line")["Mode"].to_dict()
    MUNI_full["Mode"] = MUNI_full["Line"].map(mode_dict)
    MUNI_full["Key_line_dir"] = MUNI_full["Line"].astype(str) + MUNI_full["Direction"]
    MUNI_full["Key_line_tod"] = (
        MUNI_full["Line"].astype(str) + MUNI_full["TOD"] + MUNI_full["Direction"]
    )
    MUNI_full = MUNI_full[
        [
            "Line",
            "Mode",
            "Direction",
            "TOD",
            "Key_line_dir",
            "Key_line_tod",
            "Ridership",
            "Name",
        ]
    ]
    MUNI_full = MUNI_full.sort_values(by=["Line", "Direction", "TOD"]).reset_index(
        drop=True
    )
    #MUNI_full.to_csv(output_transit_dir / "model_MUNI_Line.csv", index=False)
    return MUNI_full

In [90]:
HWY_SCREENS = {
    "SamTrans": [
        [40029, 7732, 52774, 33539, 51113, 21584, 50995],  # inbound
        [52118, 52264, 21493, 33737, 22464, 21522, 20306],  # outbound
        ["SamTrans", "Countyline", "SamTrans", "Local Bus"],
    ],
    "GG Transit": [
        [8318, 8315],  # inbound
        [8338, 8339],  # outbound
        ["Golden Gate Transit", "Golden Gate", "Golden Gate Transit", "Local Bus"],
    ],
    "GG Ferry": [
        [15503, 15608, 15503, 15608, 15502],  # inbound
        [15501, 15600, 15601, 15601, 15600],  # outbound
        ["Ferry", "Golden Gate", "Golden Gate Ferry", "Ferry"],
    ],
    "CalTrain": [
        [14659, 14659, 14661, 14660, 14661, 14660],  # inbound
        [14658, 14655, 14655, 14655, 14656, 14656],  # outbound
        ["Caltrain", "Countyline", "CalTrain", "Premium"],
    ],
    "AC transit": [
        [52833, 52832],  # inbound
        [52495, 52494],  # outbound
        ["AC Transit", "Transbay", "AC Transit", "Premium"],
    ],
}


def process_data(file_name, system, A, B, Screenline, Operator, Mode):
    # ST_IB, ST_OB: HOTFIX commented out for now; TODO remove if unused
    # # Create DataFrames for IB and OB
    # ST_IB = pd.DataFrame({"A": A, "B": B})
    # ST_OB = pd.DataFrame({"A": B, "B": A})

    # Read the DBF file and group by 'A' and 'B' while summing 'AB_VOL'
    ST_TOD = read_dbf_and_groupby_sum(file_name, system, ["A", "B", "TOD"], "AB_VOL")

    # Filter rows for IB and calculate the sum of 'AB_VOL"
    ST_TOD_IB = ST_TOD[(ST_TOD["A"].isin(A)) & (ST_TOD["B"].isin(B))]
    IB_sum = ST_TOD_IB.groupby("TOD")["AB_VOL"].sum().reset_index()
    IB_sum["Screenline"] = Screenline
    IB_sum["Direction"] = "IB"
    IB_sum["Operator"] = Operator
    IB_sum["Mode"] = Mode

    # Filter rows for OB and calculate the sum of 'AB_VOL'
    ST_TOD_OB = ST_TOD[(ST_TOD["A"].isin(B)) & (ST_TOD["B"].isin(A))]
    OB_sum = ST_TOD_OB.groupby("TOD")["AB_VOL"].sum().reset_index()
    OB_sum["Screenline"] = Screenline
    OB_sum["Direction"] = "OB"
    OB_sum["Operator"] = Operator
    OB_sum["Mode"] = Mode


    ST_TOD = pd.concat([IB_sum, OB_sum])
    ST_TOD = ST_TOD.rename(columns={"AB_VOL":"Ridership"})

    return ST_TOD


def screen_df(file_name, HWY_SCREENS):
    screenline_total = []
    for i in HWY_SCREENS.keys():
        screenline = process_data(
                file_name,
                HWY_SCREENS[i][2][0],
                HWY_SCREENS[i][0],
                HWY_SCREENS[i][1],
                HWY_SCREENS[i][2][1],
                HWY_SCREENS[i][2][2],
                HWY_SCREENS[i][2][3],
            )
        screenline["Key"] = screenline["Screenline"] + screenline["Operator"] + screenline["TOD"] + screenline["Direction"]
        screenline = screenline[
            ["Screenline", "Direction", "TOD", "Key", "Ridership", "Operator", "Mode"]
        ]
        screenline = screenline.sort_values(by="Direction").reset_index(drop=True)
        screenline_total.append(screenline)
    model_Screenlines = pd.concat(screenline_total)
    return model_Screenlines

def concat_final_SL(file_name, HWY_SCREENS, output_dir):
    model_Screenlines = screen_df(file_name,HWY_SCREENS)
    BART_Screenlines = pd.read_csv(output_dir / "model_BART_SL.csv")
    BART_Screenlines["Operator"] = "BART"
    BART_Screenlines["Mode"] = "BART"
    BART_Screenlines["Key"] = (
        BART_Screenlines["Screenline"]
        + BART_Screenlines["Operator"]
        + BART_Screenlines["TOD"]
        + BART_Screenlines["Direction"]
    )
    model_SL = pd.concat([BART_Screenlines, model_Screenlines]).reset_index(drop=True)
    model_SL.to_csv(output_dir / "model_SL.csv", index=False)
    return model_SL

In [82]:
ST_TOD = read_dbf_and_groupby_sum(data, 'SamTrans', ["A", "B", "TOD"], "AB_VOL")
a = ST_TOD.groupby("TOD")["AB_VOL"].sum().reset_index()
a

,TOD,AB_VOL
0,AM,156853.54
1,EA,6589.88
2,EV,41207.57
3,MD,178282.48
4,PM,113476.89


In [91]:
model_SL = concat_final_SL(data, HWY_SCREENS, output_transit_dir)
model_SL

,Screenline,Direction,TOD,Key,Ridership,Operator,Mode
0,Transbay,IB,AM,TransbayBARTAMIB,56079.90,BART,BART
1,Transbay,IB,EA,TransbayBARTEAIB,8072.47,BART,BART
2,Transbay,IB,EV,TransbayBARTEVIB,2513.16,BART,BART
3,Transbay,IB,MD,TransbayBARTMDIB,25334.91,BART,BART
4,Transbay,IB,PM,TransbayBARTPMIB,6487.78,BART,BART
...,...,...,...,...,...,...,...
69,Transbay,OB,AM,TransbayAC TransitAMOB,159.29,AC Transit,Premium
70,Transbay,OB,EA,TransbayAC TransitEAOB,46.53,AC Transit,Premium
71,Transbay,OB,EV,TransbayAC TransitEVOB,2280.90,AC Transit,Premium
72,Transbay,OB,MD,TransbayAC TransitMDOB,574.19,AC Transit,Premium


In [89]:
pd.read_csv(output_transit_dir / "model_BART_SL.csv")

,Screenline,Direction,TOD,Key,Ridership
0,Transbay,IB,AM,TransbayIBAM,56079.90
1,Transbay,IB,EA,TransbayIBEA,8072.47
2,Transbay,IB,EV,TransbayIBEV,2513.16
3,Transbay,IB,MD,TransbayIBMD,25334.91
4,Transbay,IB,PM,TransbayIBPM,6487.78
5,Transbay,OB,AM,TransbayOBAM,4940.79
6,Transbay,OB,EA,TransbayOBEA,663.68
7,Transbay,OB,EV,TransbayOBEV,16131.46
8,Transbay,OB,MD,TransbayOBMD,25352.75
9,Transbay,OB,PM,TransbayOBPM,49445.53


In [135]:

with open("transit.toml", "rb") as f:
    config = tomllib.load(f)

WORKING_FOLDER = output_transit_dir
OUTPUT_FOLDER = Path(config["directories"]["markdown_output_dir"])
INPUT_FOLDER = Path(config["directories"]["transit_input_dir"])
MUNI_output_dir = Path(config["directories"]["MUNI_output_dir"])
BART_output_dir = Path(config["directories"]["BART_output_dir"])
Screenline_output_dir = Path(config["directories"]["Screenline_output_dir"])
observed_BART = INPUT_FOLDER / config["transit"]["observed_BART"]
observed_BART_county = INPUT_FOLDER / config["transit"]["observed_BART_county"]
observed_BART_SL = INPUT_FOLDER / config["transit"]["observed_BART_SL"]
observed_MUNI_Line = INPUT_FOLDER / config["transit"]["observed_MUNI_Line"]
observed_SL = INPUT_FOLDER / config["transit"]["observed_SL"]
model_BART = WORKING_FOLDER / config["output"]["model_BART"]
model_BART_county = WORKING_FOLDER / config["output"]["model_BART_county"]
model_BART_SL = WORKING_FOLDER / config["output"]["model_BART_SL"]
model_MUNI_Line = WORKING_FOLDER / config["output"]["model_MUNI_Line"]
model_SL = WORKING_FOLDER / config["output"]["model_SL"]

In [102]:
observed_BART_county

WindowsPath('Q:/Model Development/CHAMP7/Validation_2023/TransitValidation/Observed/BART_county_ridership_2023.csv')

In [95]:
model_MUNI_Line

WindowsPath('X:/Projects/CHAMP7/Run25z_Market/validation_workbook/output/transit/model_MUNI_Line.csv')

In [97]:
def format_dataframe(df, numeric_columns, percentage_columns=None):
    """
    Format a DataFrame for readable display.
    - Fills NA values with '-'.
    - Formats specified numeric columns with commas and no decimal places.
    - Formats specified columns as percentages.

    Parameters:
    df (pd.DataFrame): The DataFrame to format.
    numeric_columns (list): List of numeric column names to format.
    percentage_columns (list): List of column names to format as percentages.

    Returns:
    pd.DataFrame: The formatted DataFrame.
    """
    if percentage_columns is None:
        percentage_columns = []

    # Fill NA values
    formatted_df = df.fillna("-")

    # Format specified numeric columns
    for col in numeric_columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: format_numeric(x))

    # Format percentage columns
    for col in percentage_columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: format_percentage(x))

    return formatted_df


def format_numeric(x):
    """Format a numeric value with commas and no decimal places."""
    try:
        return f"{float(x):,.0f}" if x not in ["-", ""] else x
    except ValueError:
        return x


def format_percentage(x):
    """Format a value as a percentage."""
    try:
        return f"{float(x):.0f}%" if x not in ["-", ""] else x
    except ValueError:
        return x


def convert_to_integer(value):
    try:
        return int(value)
    except ValueError:
        return value

In [98]:
def process_data(
    obs_MUNI_line,
    model_MUNI_line,
    filters,
    groupby_column,
    sum_column,
    rename_column,
    join_method,
):
    """
    Process MUNI data with flexible filtering, grouping, and summing.

    Parameters:
    obs_MUNI_line (DataFrame): The observed MUNI line data.
    model_MUNI_line (DataFrame): The modeled MUNI line data.
    filters (list of tuples): List of filter conditions, where each tuple contains (column_name, value).
    groupby_column (str): Column name to group by.
    sum_column (str): Column name to sum.
    rename_column (str): New name to rename the groupby column to in the output.

    Returns:
    DataFrame: The processed and formatted DataFrame.
    """

    # Apply the custom function to convert values
    model_MUNI_line[groupby_column] = model_MUNI_line[groupby_column].apply(
        convert_to_integer
    )
    obs_MUNI_line[groupby_column] = obs_MUNI_line[groupby_column].apply(
        convert_to_integer
    )

    # Apply filters
    if filters is not None:
        for filter_col, filter_val in filters:
            obs_MUNI_line = obs_MUNI_line[obs_MUNI_line[filter_col] == filter_val]
            model_MUNI_line = model_MUNI_line[model_MUNI_line[filter_col] == filter_val]

    # Processing observed data
    MUNI_IB_obs = obs_MUNI_line.groupby(groupby_column)[sum_column].sum().reset_index()
    MUNI_IB_obs.rename(
        columns={groupby_column: rename_column, sum_column: "Observed"}, inplace=True
    )

    # Processing modeled data
    MUNI_IB_model = (
        model_MUNI_line.groupby(groupby_column)[sum_column].sum().reset_index()
    )
    MUNI_IB_model.rename(
        columns={groupby_column: rename_column, sum_column: "Modeled"}, inplace=True
    )

    # Merging observed and modeled data
    MUNI_IB = pd.merge(MUNI_IB_obs, MUNI_IB_model, on=rename_column, how=join_method)

    # Calculating total row and appending it
    total_row = pd.Series(MUNI_IB[["Observed", "Modeled"]].sum(), name="Total")
    MUNI_IB = pd.concat([MUNI_IB, total_row.to_frame().T], ignore_index=True)

    # Calculating 'Diff' and 'Percentage Diff'
    MUNI_IB["Diff"] = MUNI_IB["Modeled"] - MUNI_IB["Observed"]
    MUNI_IB["Percentage Diff"] = MUNI_IB["Diff"] * 100 / MUNI_IB["Observed"]

    # Assign 'Total' label to the last row in the renamed column
    MUNI_IB.at[MUNI_IB.index[-1], rename_column] = "Total"

    # Formatting the DataFrame
    numeric_cols = ["Observed", "Modeled", "Diff"]
    MUNI_IB = format_dataframe(
        MUNI_IB, numeric_columns=numeric_cols, percentage_columns=["Percentage Diff"]
    )

    return MUNI_IB

In [100]:
obs_MUNI_line

,Line,Mode,Direction,TOD,Boardings
0,1,Local Bus,IB,AM,1786
1,1,Local Bus,IB,EA,108
2,1,Local Bus,IB,EV,609
3,1,Local Bus,IB,MD,4010
4,1,Local Bus,IB,PM,1623
...,...,...,...,...,...
531,F-Market & Wharves,Streetcar,IB,PM,545
532,F-Market & Wharves,Streetcar,OB,AM,116
533,F-Market & Wharves,Streetcar,OB,EV,478
534,F-Market & Wharves,Streetcar,OB,MD,1405


In [101]:
obs_MUNI_line = pd.read_csv(observed_MUNI_Line)
model_MUNI_line = pd.read_csv(model_MUNI_Line)
tod_order = ["EA", "AM", "MD", "PM", "EV", "Total"]
MUNI_IB = process_data(
    obs_MUNI_line,
    model_MUNI_line,
    [("Direction", "IB")],
    "Line",
    "Ridership",
    "Route",
    "left",
)

In [115]:
obs_BART_county = pd.read_csv(observed_BART_county)
model_BART_county = pd.read_csv(model_BART_county)
county_order = ["San Francisco", "San Mateo", "Santa Clara", "Contra Costa", "Alameda", "Total"]
county_br_day = process_data(
    obs_BART_county,
    model_BART_county,
    None,
    "County",
    "Boardings",
    "County",
    "left",
)
county_br_day["County"] = pd.Categorical(
    county_br_day["County"], categories=county_order, ordered=True
)

In [111]:
obs_BART_county

,County,TOD,Key,Boardings,Alightings
0,Alameda,AM,AlamedaAM,42572.0,25365.0
1,Alameda,EA,AlamedaEA,2242.0,1267.0
2,Alameda,EV,AlamedaEV,22878.0,34128.0
3,Alameda,MD,AlamedaMD,52818.0,42107.0
4,Alameda,PM,AlamedaPM,35182.0,51009.0
5,Contra Costa,AM,Contra CostaAM,26740.0,4832.0
6,Contra Costa,EA,Contra CostaEA,1712.0,266.0
7,Contra Costa,EV,Contra CostaEV,5179.0,16364.0
8,Contra Costa,MD,Contra CostaMD,21117.0,13424.0
9,Contra Costa,PM,Contra CostaPM,7338.0,26697.0


In [136]:
obs_SL = pd.read_csv(observed_SL)
# obs_SL["Ridership"] = (
#     obs_SL["Ridership"]
#     .replace({"-": "0", " -   ": "0"})
#     .str.replace(",", "")
#     .astype(float)
# )
model_SL = pd.read_csv(model_SL)
transbay_AC_IB = process_data(
    obs_SL,
    model_SL,
    [("Screenline", "Transbay"), ("Operator", "AC Transit"), ("Direction", "IB")],
    "TOD",
    "Ridership",
    "TOD",
    "left",
)

In [121]:
transbay_AC_IB = process_data(
    obs_SL,
    model_SL,
    [("Screenline", "Transbay"), ("Operator", "AC Transit"), ("Direction", "IB")],
    "TOD",
    "Ridership",
    "TOD",
    "left",
)
transbay_AC_IB["TOD"] = pd.Categorical(
    transbay_AC_IB["TOD"], categories=tod_order, ordered=True
)
transbay_AC_IB = transbay_AC_IB.sort_values("TOD")

In [123]:
transbay_AC_OB = process_data(
    obs_SL,
    model_SL,
    [("Screenline", "Transbay"), ("Operator", "AC Transit"), ("Direction", "OB")],
    "TOD",
    "Ridership",
    "TOD",
    "right",
)
transbay_AC_OB["TOD"] = pd.Categorical(
    transbay_AC_OB["TOD"], categories=tod_order, ordered=True
)
transbay_AC_OB = transbay_AC_OB.sort_values("TOD")

In [145]:
Countyline_CalTrain_IB = process_data(
    obs_SL,
    model_SL,
    [("Screenline", "Countyline"), ("Operator", "CalTrain"), ("Direction", "IB")],
    "TOD",
    "Ridership",
    "TOD",
    "right",
)
Countyline_CalTrain_IB["TOD"] = pd.Categorical(
    Countyline_CalTrain_IB["TOD"], categories=tod_order, ordered=True
)
Countyline_CalTrain_IB = Countyline_CalTrain_IB.sort_values("TOD")

In [146]:
obs_SL[obs_SL["Operator"]=="Caltrain"]

,Screenline,Direction,TOD,Ridership,Operator,Mode,Key
48,Countyline,IB,AM,2678,Caltrain,Premium,CountylineCaltrainAMIB
49,Countyline,IB,EV,442,Caltrain,Premium,CountylineCaltrainEVIB
50,Countyline,IB,MD,694,Caltrain,Premium,CountylineCaltrainMDIB
51,Countyline,IB,PM,1379,Caltrain,Premium,CountylineCaltrainPMIB
52,Countyline,OB,AM,898,Caltrain,Premium,CountylineCaltrainAMOB
53,Countyline,OB,EV,948,Caltrain,Premium,CountylineCaltrainEVOB
54,Countyline,OB,MD,1017,Caltrain,Premium,CountylineCaltrainMDOB
55,Countyline,OB,PM,1980,Caltrain,Premium,CountylineCaltrainPMOB


In [147]:
Countyline_CalTrain_IB

,TOD,Observed,Modeled,Diff,Percentage Diff
0,Total,0,0,0,-


In [118]:
obs_SL

,Screenline,Direction,TOD,Ridership,Operator,Mode,Key
0,Countyline,IB,AM,8873,BART,BART,CountylineBARTAMIB
1,Countyline,IB,EA,439,BART,BART,CountylineBARTEAIB
2,Countyline,IB,EV,6098,BART,BART,CountylineBARTEVIB
3,Countyline,IB,MD,12278,BART,BART,CountylineBARTMDIB
4,Countyline,IB,PM,7364,BART,BART,CountylineBARTPMIB
...,...,...,...,...,...,...,...
60,Transbay,OB,PM,453,AC Transit,Premium,TransbayAC TransitPMOB
61,Transbay,IB,AM,394,AC Transit,Premium,TransbayAC TransitAMIB
62,Transbay,IB,EV,666,AC Transit,Premium,TransbayAC TransitEVIB
63,Transbay,IB,MD,1389,AC Transit,Premium,TransbayAC TransitMDIB


In [138]:
model_BART_county

WindowsPath('X:/Projects/CHAMP7/Run25z_Market/validation_workbook/output/transit/model_BART_county.csv')

In [139]:
county_br_day

,County,Observed,Modeled,Diff,Percentage Diff
0,Alameda,"155,692","158,776","3,084",2%
1,Contra Costa,"62,086","54,650","-7,436",-12%
2,San Francisco,"176,087","164,980","-11,107",-6%
3,San Mateo,"42,208","29,776","-12,432",-29%
4,Santa Clara,"7,050",-,-,-
5,Total,"443,123","408,181","-34,942",-8%
